#**Back Propagation**

#Step 1 : import libraries

In [158]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#Step 2 :Load the Dataset
###Upload palmer penguins dataset " / LAB/ Dataset/ penguins.csv





In [159]:
# Loading dataset

penguins= sns.load_dataset("penguins")

In [160]:
# Dividing the dataset into target variable and features
penguins = sns.load_dataset("penguins")
X = penguins.drop(columns=['species'])  #features
y = penguins['species'] #target


In [161]:
penguins .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [162]:
X.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Torgersen,36.7,19.3,193.0,3450.0,Female


In [163]:
penguins .isna().sum()

,0
species,0
island,0
bill_length_mm,2
bill_depth_mm,2
flipper_length_mm,2
body_mass_g,2
sex,11


In [164]:
penguins .shape

(344, 7)

In [165]:
penguins .dropna(inplace=True)#Drop all value

In [166]:
#Encoding
le = LabelEncoder()
X['island'] = le.fit_transform(X['island'])
X['sex'] = le.fit_transform(X['sex'])


y = le.fit_transform(y)

In [167]:
X.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,2,39.1,18.7,181.0,3750.0,1
1,2,39.5,17.4,186.0,3800.0,0
2,2,40.3,18.0,195.0,3250.0,0
3,2,NaN,NaN,NaN,NaN,2
4,2,36.7,19.3,193.0,3450.0,0


In [168]:
# Standard scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

#Step 3: Split Dataset in Training and Testing


In [169]:
# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [170]:
y_train.size

275

In [171]:
y_train.shape

(275,)

In [172]:
X_train.shape

(275, 6)

In [173]:
np.unique(y)

array([0, 1, 2])

# Step 4 : initializing the hyperparameters

In [174]:
learning_rate = 0.1
iterations = 5000
N = y_train.size

# Input features
input_size = 6

# Hidden layers
hidden_size = 4

# Output layer
output_size = 6


#Step 5 : Initialize Weights

In [175]:
np.random.seed(10)

# Hidden layer
W1 = np.random.normal(scale=0.5, size=(input_size, hidden_size))

# Output layer
W2 = np.random.normal(scale=0.5, size=(hidden_size , output_size))


#Step 6 : mean squared error, accuracy and sigmoid.



In [176]:
# Helper functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(y_pred, y_true):
    # One-hot encode y_true (i.e., convert [0, 1, 2] into [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    y_true_one_hot = np.eye(output_size)[y_true]

    # Reshape y_true_one_hot to match y_pred shape
    y_true_reshaped = y_true_one_hot.reshape(y_pred.shape)

    # Compute the mean squared error between y_pred and y_true_reshaped
    error = ((y_pred - y_true_reshaped)**2).sum() / (2*y_pred.size)

    return error

def accuracy(y_pred, y_true):
    acc = y_pred.argmax(axis=1) ==  y_true.argmax(axis=1)
    return acc.mean()

results = pd.DataFrame(columns=["mse", "accuracy"])
print(np.unique(y_train))

['Adelie' 'Chinstrap' 'Gentoo']


#Step 7 : Building the Backpropogation Model in Python
###We will create a for loop for a given number of iterations and will update the weights in each iteration. The model will go through three phases feedforward propagation, the error calculation phase, and the backpropagation phase.


In [177]:
# Training loop

for itr in range(iterations):
    # Feedforward propagation
    X_train_numeric = X_train.select_dtypes(include=np.number).astype(np.float32)
    #X_train_numeric = X_train.values.astype(np.float32)
    #Z1 = np.dot(X_train.values, W1)
    Z1 = np.dot(X_train_numeric, W1)
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2)
    A2 = sigmoid(Z2)

    # Calculate error
    mse = mean_squared_error(A2, y_train)
    acc = accuracy(np.eye(output_size)[y_train], A2)
    new_row = pd.DataFrame({"mse": [mse], "accuracy": [acc]})
    results = pd.concat([results, new_row], ignore_index=True)

    # Backpropagation
    E1 = A2 - np.eye(output_size)[y_train]
    dW1 = E1 * A2 * (1 - A2)
    E2 = np.dot(dW1, W2.T)
    dW2 = E2 * A1 * (1 - A1)

    # Update weights
    W2_update = np.dot(A1.T, dW1) / N
    #W1_update = np.dot(X_train.values.T, dW2) / N
    W1_update = np.dot(X_train.values.astype(np.float32).T, dW2) / N
    W2 = W2 - learning_rate * W2_update
    W1 = W1 - learning_rate * W1_update

ValueError: shapes (275,4) and (6,4) not aligned: 4 (dim 1) != 6 (dim 0)

# Step 8 : Plot the mean squared error and accuracy using the pandas plot() function.



In [180]:
results.mse.plot(title="Mean Squared Error")
plt.show()
results.accuracy.plot(title="Accuracy")
plt.show()


TypeError: no numeric data to plot

###Now we will calculate the accuracy of the model.



In [181]:
# Test the model

Z1 = np.dot(X_test, W1)
A1 = sigmoid(Z1)
Z2 = np.dot(A1, W2)
A2 = sigmoid(Z2)
test_acc = accuracy(np.eye(output_size)[y_test], A2)
print("Test accuracy: {}".format(test_acc))


TypeError: can't multiply sequence by non-int of type 'float'

#**Well Done :)**